# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
cities_df = pd.read_csv('weather_output.csv')
cities_df.dropna(inplace = True) 
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Champerico,14.3000,-91.9167,64.00,93,0,7.07,GT,1612662661
2,Tokonou,9.6500,-9.7833,71.20,23,22,3.69,GN,1612662732
3,Tsimmerbude,54.6750,20.1347,21.99,85,0,2.24,RU,1612662733
4,Chadiza,-14.0678,32.4392,64.38,98,100,1.07,ZM,1612662733
5,Vanimo,-2.6741,141.3028,82.62,78,99,4.07,PG,1612662735


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
cities_df.count()
cities_df["Humidity"].max()


100

In [36]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [37]:
# Store latitude and longitude in locations
locations = cities_df[["Lat","Lng"]]

# Fill NaN values and convert to int
humidity = cities_df["Humidity"]

In [38]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                     point_radius=5)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) \
                                   & (cities_df["Cloudiness"] == 0) \
                                   & (cities_df["Wind Speed"] > 10)].dropna()
# narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.reset_index(inplace=True)
del narrowed_cities_df['index']
narrowed_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cockburn Town,21.4612,-71.1419,77.47,78,0,23.31,TC,1612662669
1,Hong Kong,22.2855,114.1577,73.00,46,0,11.01,HK,1612662757
2,General Pico,-35.6566,-63.7568,74.77,33,0,11.30,AR,1612662761
3,Marsh Harbour,26.5412,-77.0636,75.07,85,0,17.94,BS,1612662775
4,Barra Patuca,15.8000,-84.2833,78.33,83,0,14.97,HN,1612662792
5,Tumut,-35.3000,148.2167,77.00,47,0,12.66,AU,1612662801
6,Bariloche,-41.1456,-71.3082,75.20,29,0,11.50,AR,1612662804


## Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = []

for hotel in range(len(narrowed_cities_df)):
    lat = narrowed_cities_df.loc[hotel]['Lat']
    lng = narrowed_cities_df.loc[hotel]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotels = response.json()
    try:
        hotel_df.append(hotels['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_cities_df["Hotel Name"] = hotel_df
narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Cockburn Town,21.4612,-71.1419,77.47,78,0,23.31,TC,1612662669,Cockburn Town
1,Hong Kong,22.2855,114.1577,73.00,46,0,11.01,HK,1612662757,Hong Kong
2,General Pico,-35.6566,-63.7568,74.77,33,0,11.30,AR,1612662761,General Pico
3,Marsh Harbour,26.5412,-77.0636,75.07,85,0,17.94,BS,1612662775,Marsh Harbour
4,Barra Patuca,15.8000,-84.2833,78.33,83,0,14.97,HN,1612662792,Barra Patuca


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_cities_df.iterrows()]
locations = narrowed_cities_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in narrowed_cities_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…